In [1]:
path_model_to_eval = (
    "experiments/cross_stitch/2024-08-28_09-27-03/models/best_model_epoch_50.pth"
)

In [2]:
import os
import sys
import argparse

print(os.getcwd())
os.chdir("../../")
os.getcwd()

sys.argv = [
    "view",
    "--config",
    "config/multi_task_cross_stitch.yaml",
    "--model_path",
    path_model_to_eval,
]


parser = argparse.ArgumentParser()
parser.add_argument("--config", type=str, required=True, help="Path to the config file")
parser.add_argument(
    "--model_path", type=str, required=True, help="Path to the model pth"
)

args = parser.parse_args()

print(args.config)
print(args.model_path)

/data01/dl23vitcas/dl_project/src/multi_task_cross_stitch
config/multi_task_cross_stitch.yaml
experiments/cross_stitch/2024-08-28_09-27-03/models/best_model_epoch_50.pth


In [3]:
import torch
from config_experiments import config, parse_args
from utils import set_seed, set_device
from dataloader import VOC08Attr
import torchvision.transforms as transforms
from model import ObjectDetectionModel, AttributePredictionModel, CrossStitchNet
from metrics import (
    compute_mAP_obj_detect,
    compute_mAP_attr_predict,
)

import wandb

In [4]:
def view_mAP_for_class(mAP, data_test):
    print(f"\nmAP@0.50 (per class):")
    index = torch.arange(1, config["global"]["num_classes"] + 1)

    for i, value in zip(index, mAP["map_per_class"].numpy()):
        category = data_test.id2category.get(i.item())
        mAP_category = value.item()

        print(f"\tAP {category} : {(mAP_category):.2f}")
        wandb.config.update({f"AP {category} ": mAP_category})

    mAP50 = mAP["map_50"].item()
    print(f"\nmAP@0.50 : {mAP50:.2f}")

    wandb.config.update({"mAP@0.50": mAP50})


def mAP_view_attributes(mAP, data_test):
    print(mAP)
    print(f"\nmAP@0.50 (per class):")
    for i, value in enumerate(mAP):
        attribute = data_test.id2attribute[i + 1]
        print(f"AP {attribute}: {value:.2f}")
        wandb.config.update({f"AP {attribute} ": value})
    mAP = torch.mean(mAP)

    print(f"\nmAP : {mAP:.2f}")

    wandb.config.update({"mAP_attr@0.50": mAP})

In [5]:
model_path = parse_args().model_path
transform_test = transforms.Compose(
    [
        transforms.Resize(size=config["transform"]["resize_values"]),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=config["transform"]["mean"], std=config["transform"]["std"]
        ),
    ]
)
set_seed(config["global"]["seed"])
device = set_device(config["global"]["gpu_id"])

wandb.init(
    group="cross_stitch",
    project="DL",
    config=config,
    save_code=True,
    mode="disabled",
)

data_test = VOC08Attr(train=False, transform=transform_test)

model = CrossStitchNet(
    ObjectDetectionModel().to(device).backbone,
    AttributePredictionModel().to(device).backbone,
).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<All keys matched successfully>

In [6]:
mAP = compute_mAP_obj_detect(data_test, model, device)
view_mAP_for_class(mAP, data_test)

Compute mAP: 100%|██████████| 2227/2227 [14:50<00:00,  2.50it/s]



mAP@0.50 (per class):
	AP horse : 0.67
	AP person : 0.53
	AP bottle : 0.22
	AP dog : 0.65
	AP tvmonitor : 0.55
	AP car : 0.46
	AP aeroplane : 0.55
	AP bicycle : 0.49
	AP boat : 0.21
	AP chair : 0.23
	AP diningtable : 0.30
	AP pottedplant : 0.20
	AP train : 0.65
	AP cat : 0.68
	AP sofa : 0.26
	AP bird : 0.43
	AP sheep : 0.34
	AP motorbike : 0.62
	AP bus : 0.38
	AP cow : 0.35

mAP@0.50 : 0.44


In [7]:
mAP = compute_mAP_attr_predict(data_test, model, device)
mAP_view_attributes(mAP, data_test)

Compute mAP: 100%|██████████| 2227/2227 [04:30<00:00,  8.24it/s]


tensor([0.3297, 0.5857, 0.2522, 0.7700, 0.5375, 0.5488, 0.6866, 0.7294, 0.8696,
        0.7974, 0.8580, 0.8198, 0.8077, 0.7852, 0.8060, 0.8071, 0.7717, 0.7034,
        0.7959, 0.8125, 0.7211, 0.7932, 0.3445, 0.6528, 0.8050, 0.7136, 0.8089,
        0.6562, 0.5499, 0.4371, 0.6107, 0.2699, 0.8157, 0.8212, 0.6169, 0.5841,
        0.6762, 0.5488, 0.8131, 0.6569, 0.7509, 0.6977, 0.5179, 0.2413, 0.5604,
        0.5781, 0.8639, 0.7095, 0.6523, 0.8648, 0.8982, 0.8097, 0.8494, 0.5089,
        0.6373, 0.8485, 0.8903, 0.7636, 0.7854, 0.6745, 0.3488, 0.4545, 0.8617,
        0.1109], device='cuda:0')

mAP@0.50 (per class):
AP 2D Boxy: 0.33
AP 3D Boxy: 0.59
AP Round: 0.25
AP Vert Cyl: 0.77
AP Horiz Cyl: 0.54
AP Occluded: 0.55
AP Tail: 0.69
AP Beak: 0.73
AP Head: 0.87
AP Ear: 0.80
AP Snout: 0.86
AP Nose: 0.82
AP Mouth: 0.81
AP Hair: 0.79
AP Face: 0.81
AP Eye: 0.81
AP Torso: 0.77
AP Hand: 0.70
AP Arm: 0.80
AP Leg: 0.81
AP Foot/Shoe: 0.72
AP Wing: 0.79
AP Propeller: 0.34
AP Jet engine: 0.65
AP Window: 0